# Model Prediction

In [4]:
import csv
from datetime import datetime, timedelta
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode

MODEL_PATH = "data/lstm.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(TRAIN_TEST_RATE)

times = [int(item['openTime']) for item in reader]
times = times[int(len(times) * TRAIN_TEST_RATE) + 4:]
times = [datetime.utcfromtimestamp(t / 1000.0) for t in times]
# times.extend([times[-1] + timedelta(days=n) for n in range(1, 31)])

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(X_test)
pred = machine.scaler.inverse_transform(pred)

Y_test = machine.scaler.inverse_transform(Y_test)

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pd.DataFrame({
        'Predictions': [item[c] for item in pred],
        'Actual': [item[c] for item in Y_test]
    }, index=times).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )